10 The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [1]:
spark

org.apache.spark.sql.SparkSession@729ab107

In [3]:
import org.apache.spark.sql.functions._

In [4]:
import org.apache.spark.sql.types._

In [5]:
import org.apache.spark.sql.expressions.Window

In [7]:
val spark=SparkSession.builder().appName("movies app").master("local[*]").getOrCreate()

spark = org.apache.spark.sql.SparkSession@3c876492


org.apache.spark.sql.SparkSession@3c876492

In [8]:
import spark.implicits._

UDFs

In [9]:
val getYear=udf((year:String)=>{
    val p="\\d{4}".r
    val yr=p.findFirstMatchIn(year).getOrElse("").toString
    if(yr!=""){
        yr.toInt
    }
    else{
        0
    }
})

getYear = UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))

In [10]:
val isLeapYear=udf((year:String)=>{
    val p="\\d{4}".r
    val yr=p.findFirstMatchIn(year).getOrElse("").toString
    if(yr!=""){
        val y:Int=yr.toInt
        if(y%4==0 && y%100!=0){
            true
        }
        else if(y%400==0){
            true
        }
        else{
            false
        }
    }
    else{
        false
    }
})

isLeapYear = UserDefinedFunction(<function1>,BooleanType,Some(List(StringType)))


UserDefinedFunction(<function1>,BooleanType,Some(List(StringType)))

loading movies

In [10]:
val Movie=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Movie.csv")
.dropDuplicates()

Movie = [index: string, MID: string ... 4 more fields]


[index: string, MID: string ... 4 more fields]

printing table structure to know what are the columns and it's data type

In [11]:
Movie.printSchema

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- num_votes: string (nullable = true)



Displaying first two records to see sample data

observation 1: year,rating,num_votes,index columns data types should be Integer

observation 2: also we need do trim to remove any extra spaces

In [12]:
Movie.show(2)

+-----+---------+-----------------+----+------+---------+
|index|      MID|            title|year|rating|num_votes|
+-----+---------+-----------------+----+------+---------+
|   85|tt3224288|Beyond the Clouds|2017|   7.0|     1123|
|  913|tt2178508|   Son of Sardaar|2012|   4.0|     7895|
+-----+---------+-----------------+----+------+---------+
only showing top 2 rows



displaying total records in the table

In [13]:
Movie.count

3475

as per observation need to change the data type

In [11]:
val movies=Movie.withColumn("year",getYear(Movie("year")).cast(IntegerType))
.withColumn("title",trim(Movie("title")))
.withColumn("MID",trim(Movie("MID")))
.withColumn("rating",trim(Movie("rating")).cast(DoubleType))
.withColumn("num_votes",trim(Movie("num_votes")).cast(LongType))
.withColumn("index",trim(Movie("index")).cast(IntegerType))
.dropDuplicates("MID")

movies = [index: int, MID: string ... 4 more fields]


[index: int, MID: string ... 4 more fields]

now print the schema to verify columns and it's data type

In [15]:
movies.printSchema

root
 |-- index: integer (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = false)
 |-- rating: double (nullable = true)
 |-- num_votes: long (nullable = true)



print the total number of records in the table after data cleaning

In [16]:
movies.count

3475

show the first two records as a sample data

In [17]:
movies.show(2)

+-----+---------+--------------------+----+------+---------+
|index|      MID|               title|year|rating|num_votes|
+-----+---------+--------------------+----+------+---------+
| 3008|tt0036077|              Kismet|1943|   7.5|       71|
| 2969|tt0102701|Prahaar: The Fina...|1991|   7.9|     1696|
+-----+---------+--------------------+----+------+---------+
only showing top 2 rows



Load the Person.csv file and it has header and store theses records in Person variable

In [12]:
val Person=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Person.csv")

Person = [index: string, PID: string ... 2 more fields]


[index: string, PID: string ... 2 more fields]

Print the schema of Person variable,first two records and it's count

In [19]:
Person.show(2)
Person.printSchema
val p1Count=Person.count

+-----+---------+---------------+------+
|index|      PID|           Name|Gender|
+-----+---------+---------------+------+
|    0|nm0000288| Christian Bale|  Male|
|    1|nm0000949| Cate Blanchett|Female|
+-----+---------+---------------+------+
only showing top 2 rows

root
 |-- index: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p1Count = 38285


38285

Data Cleaning

change the data types as per data and also remove extra spaces from data

In [13]:
val persons=Person.withColumn("PID",trim(Person("PID")))
.withColumn("Name",trim(Person("Name")))
.withColumn("Gender",trim(Person("Gender")))
.withColumn("index",trim(Person("index")).cast(IntegerType))
.dropDuplicates("PID")

persons = [index: int, PID: string ... 2 more fields]


[index: int, PID: string ... 2 more fields]

show the sample data, schema and it's count of variable persons

In [21]:
persons.show(2)
persons.printSchema
val p2Count=persons.count

+-----+---------+-------------+------+
|index|      PID|         Name|Gender|
+-----+---------+-------------+------+
|15793|nm0022343|       Alpana|Female|
| 3121|nm0028416|Martin Andris|  Male|
+-----+---------+-------------+------+
only showing top 2 rows

root
 |-- index: integer (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p2Count = 37566


37566

print the difference between Person and persons records count

In [23]:
val diff=p1Count-p2Count
println(diff+" duplicates found")

719 duplicates found


diff = 719


719

Load the M_Director.csv file and it's has header

In [14]:
val M_Director=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/M_Director.csv")

M_Director = [index: string, MID: string ... 2 more fields]


[index: string, MID: string ... 2 more fields]

display sample data, schema and count M_Director variable

In [25]:
M_Director.printSchema
M_Director.show(2)
val m_d1Count=M_Director.count

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: string (nullable = true)

+-----+---------+---------+---+
|index|      MID|      PID| ID|
+-----+---------+---------+---+
|    0|tt2388771|nm0785227|  0|
|    1|tt5164214|nm0002657|  1|
+-----+---------+---------+---+
only showing top 2 rows



m_d1Count = 3475


3475

Data cleaning for M_Director

remove duplicate records, change data type from string to appropriate data type and perform trim operation on each column separately

In [15]:
val m_directors=M_Director.withColumn("ID",trim(col("ID")).cast(LongType))
.withColumn("index",trim(col("index")).cast(LongType))
.withColumn("MID",trim(col("MID")))
.withColumn("PID",trim(col("PID")))
.dropDuplicates("MID","PID")
.dropDuplicates("ID")
.dropDuplicates

m_directors = [index: bigint, MID: string ... 2 more fields]


[index: bigint, MID: string ... 2 more fields]

display sample data, schema and count m_director variable

In [27]:
m_directors.printSchema
m_directors.show(2)
val m_d2Count=m_directors.count

root
 |-- index: long (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: long (nullable = true)

+-----+---------+---------+---+
|index|      MID|      PID| ID|
+-----+---------+---------+---+
|   26|tt5074352|nm4318159| 26|
|   29|tt1098327|nm0939128| 29|
+-----+---------+---------+---+
only showing top 2 rows



m_d2Count = 3475


3475

Display Count difference between M_Director and m_director variables

In [28]:
val diff_m_directors_count=m_d1Count-m_d2Count
println(diff_m_directors_count+" d")

0 d


diff_m_directors_count = 0


0

Loading and cleaning M_Cast table

In [16]:
val M_Cast=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/M_Cast.csv")

M_Cast = [index: string, MID: string ... 2 more fields]


[index: string, MID: string ... 2 more fields]

Sample data from M_Cast table

In [30]:
M_Cast.printSchema
M_Cast.show(2)
val m_c1Count=M_Cast.count

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: string (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|    0|tt2388771| nm0000288|  0|
|    1|tt2388771| nm0000949|  1|
+-----+---------+----------+---+
only showing top 2 rows



m_c1Count = 82837


82837

Cleaning data m_cast

In [17]:
val m_cast=M_Cast.withColumn("ID",trim(col("ID")).cast(LongType))
.withColumn("index",trim(col("index")).cast(LongType))
.withColumn("MID",trim(col("MID")))
.withColumn("PID",trim(col("PID")))
.dropDuplicates("MID","PID")
//.dropDuplicates("MID","PID","ID")
.dropDuplicates("ID")
.dropDuplicates()

m_cast = [index: bigint, MID: string ... 2 more fields]


[index: bigint, MID: string ... 2 more fields]

Sample data after data cleaning for m_cast

In [32]:
m_cast.printSchema
m_cast.show(2)
val m_c2Count=m_cast.count

root
 |-- index: long (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: long (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|   26|tt2388771|nm10302077| 26|
|   29|tt2388771|nm10302080| 29|
+-----+---------+----------+---+
only showing top 2 rows



m_c2Count = 82837


82837

Print difference of total count after and before data cleaning

In [33]:
val diff_m_cast_count=m_c1Count-m_c2Count
println(diff_m_cast_count+" d")

0 d


diff_m_cast_count = 0


0

now modifying column name from PID to MPID of m_director (To avoid conflicts while joining another, if we have same column name)

In [18]:
val m_dirs=m_directors.withColumnRenamed("PID","DPID")

m_dirs = [index: bigint, MID: string ... 2 more fields]


[index: bigint, MID: string ... 2 more fields]

step 1: join the Directors with Cast table, to get the director and his cast in a row and show sample data

In [35]:
val dir_mvs=m_dirs.join(m_cast,"MID").select("MID","DPID","PID")
dir_mvs.show(5)

+---------+---------+----------+
|      MID|     DPID|       PID|
+---------+---------+----------+
|tt2388771|nm0785227|nm10302077|
|tt2388771|nm0785227|nm10302080|
|tt0848228|nm0923736| nm3510000|
|tt1010048|nm0000965| nm3193591|
|tt5970844|nm1715502| nm7961730|
+---------+---------+----------+
only showing top 5 rows



dir_mvs = [MID: string, DPID: string ... 1 more field]


[MID: string, DPID: string ... 1 more field]

step2: writing udf function to get the Yash count in a separate column after group by director and person ID's with count

In [36]:
val getYashCount=udf((dpid:String,count:Long)=>{
    if(dpid=="nm0007181"){
        Some(count)
    }
    else{
        Some(0)
    }
}:Option[Long])

getYashCount = UserDefinedFunction(<function2>,LongType,Some(List(StringType, LongType)))


UserDefinedFunction(<function2>,LongType,Some(List(StringType, LongType)))

In [37]:
val makeItNullable=udf((count:Long)=>{
        Some(count)
}:Option[Long])

makeItNullable = UserDefinedFunction(<function1>,LongType,Some(List(LongType)))


UserDefinedFunction(<function1>,LongType,Some(List(LongType)))

In [44]:
//val dir_pid_count=dir_mvs.groupBy("DPID","PID").count()
//val dir_pid_count=dir_mvs.groupBy("DPID","PID").max().filter($"DPID"==="nm0007181").select("PID")
val dir_pid_grp_by=dir_mvs.groupBy("DPID","PID");
val dir_pid_count2=dir_pid_grp_by.count()
val dir_pid_count=dir_pid_grp_by.max().filter($"DPID"==="nm0007181").select("PID")
//.withColumn("yash_count",getYashCount(col("DPID"),col("count")))
//.withColumn("count",makeItNullable(col("count")))
//dir_pid_count.count()
dir_pid_count.printSchema
dir_pid_count.sort("PID").show()

root
 |-- PID: string (nullable = true)

+---------+
|      PID|
+---------+
|nm0000821|
|nm0002043|
|nm0004109|
|nm0004334|
|nm0004429|
|nm0004434|
|nm0004435|
|nm0004437|
|nm0004487|
|nm0004564|
|nm0004570|
|nm0006348|
|nm0006369|
|nm0006433|
|nm0006689|
|nm0006762|
|nm0007106|
|nm0007147|
|nm0007181|
|nm0012881|
+---------+
only showing top 20 rows



dir_pid_grp_by = RelationalGroupedDataset: [grouping expressions: [DPID: string, PID: string], value: [MID: string, DPID: string ... 1 more field], type: GroupBy]
dir_pid_count2 = [DPID: string, PID: string ... 1 more field]
dir_pid_count = [PID: string]


[PID: string]

In [54]:
val maxCountWithDup=dir_pid_count2.withColumn("max",max("count") over Window.partitionBy("PID"))

maxCountWithDup = [DPID: string, PID: string ... 2 more fields]


[DPID: string, PID: string ... 2 more fields]

In [55]:
val maxCount=maxCountWithDup.filter($"max"===$"count")

maxCount = [DPID: string, PID: string ... 2 more fields]


[DPID: string, PID: string ... 2 more fields]

In [57]:
maxCount.sort("PID").show(100)

+---------+---------+-----+---+
|     DPID|      PID|count|max|
+---------+---------+-----+---+
|nm4998666|     null|    1|  1|
|nm3769883|     null|    1|  1|
|nm0496746|nm0000002|    1|  1|
|nm0000180|nm0000027|    1|  1|
|nm0896533|nm0000039|    1|  1|
|nm0896533|nm0000042|    1|  1|
|nm0004292|nm0000047|    1|  1|
|nm0485943|nm0000073|    1|  1|
|nm0000229|nm0000076|    1|  1|
|nm0178997|nm0000092|    1|  1|
|nm0000269|nm0000093|    1|  1|
|nm0149446|nm0000096|    1|  1|
|nm0113819|nm0000096|    1|  1|
|nm0001060|nm0000101|    1|  1|
|nm0583292|nm0000112|    1|  1|
|nm0002657|nm0000113|    1|  1|
|nm0000386|nm0000131|    1|  1|
|nm0004072|nm0000137|    1|  1|
|nm0394280|nm0000140|    1|  1|
|nm0012541|nm0000144|    1|  1|
|nm0001162|nm0000147|    1|  1|
|nm1287649|nm0000155|    1|  1|
|nm0923736|nm0000168|    1|  1|
|nm0204628|nm0000173|    1|  1|
|nm0394280|nm0000174|    1|  1|
|nm0001060|nm0000187|    1|  1|
|nm0431918|nm0000193|    1|  1|
|nm0634782|nm0000193|    1|  1|
|nm00275

In [58]:
val maxMore=maxCount.withColumn("duplicates_count",count("PID") over Window.partitionBy("PID"))

maxMore = [DPID: string, PID: string ... 3 more fields]


[DPID: string, PID: string ... 3 more fields]

In [59]:
maxMore.sort("PID").show(100)

+---------+---------+-----+---+----------------+
|     DPID|      PID|count|max|duplicates_count|
+---------+---------+-----+---+----------------+
|nm4998666|     null|    1|  1|               0|
|nm3769883|     null|    1|  1|               0|
|nm0496746|nm0000002|    1|  1|               1|
|nm0000180|nm0000027|    1|  1|               1|
|nm0896533|nm0000039|    1|  1|               1|
|nm0896533|nm0000042|    1|  1|               1|
|nm0004292|nm0000047|    1|  1|               1|
|nm0485943|nm0000073|    1|  1|               1|
|nm0000229|nm0000076|    1|  1|               1|
|nm0178997|nm0000092|    1|  1|               1|
|nm0000269|nm0000093|    1|  1|               1|
|nm0113819|nm0000096|    1|  1|               2|
|nm0149446|nm0000096|    1|  1|               2|
|nm0001060|nm0000101|    1|  1|               1|
|nm0583292|nm0000112|    1|  1|               1|
|nm0002657|nm0000113|    1|  1|               1|
|nm0000386|nm0000131|    1|  1|               1|
|nm0004072|nm0000137

In [67]:
val finalPIDwithDPID=maxMore.filter($"duplicates_count"===1 && $"DPID"==="nm0007181")

finalPIDwithDPID = [DPID: string, PID: string ... 3 more fields]


[DPID: string, PID: string ... 3 more fields]

In [69]:
finalPIDwithDPID.sort("PID").show(100)

+---------+---------+-----+---+----------------+
|     DPID|      PID|count|max|duplicates_count|
+---------+---------+-----+---+----------------+
|nm0007181|nm0004434|    7|  7|               1|
|nm0007181|nm0007181|    2|  2|               1|
|nm0007181|nm0015296|    1|  1|               1|
|nm0007181|nm0101508|    1|  1|               1|
|nm0007181|nm0158332|    3|  3|               1|
|nm0007181|nm0159158|    1|  1|               1|
|nm0007181|nm0159165|    3|  3|               1|
|nm0007181|nm0347901|    5|  5|               1|
|nm0007181|nm0348507|    1|  1|               1|
|nm0007181|nm0407002|    9|  9|               1|
|nm0007181|nm0471443|   10| 10|               1|
|nm0007181|nm0557645|    2|  2|               1|
|nm0007181|nm0576489|    1|  1|               1|
|nm0007181|nm0622730|    1|  1|               1|
|nm0007181|nm0707271|   11| 11|               1|
|nm0007181|nm0716851|    5|  5|               1|
|nm0007181|nm0755087|    4|  4|               1|
|nm0007181|nm0802183

In [73]:
val final_PIDs=finalPIDwithDPID.select("PID","count")

final_PIDs = [PID: string, count: bigint]


[PID: string, count: bigint]

In [79]:
val finalActors=final_PIDs.join(persons,"PID").dropDuplicates().select("PID","Name","Gender","count")

finalActors = [PID: string, Name: string ... 2 more fields]


[PID: string, Name: string ... 2 more fields]

# Final Answer to the problem

In [80]:
finalActors.show(100)

+---------+--------------------+------+-----+
|      PID|                Name|Gender|count|
+---------+--------------------+------+-----+
|nm1525006|     Sandow S. Sethi|  Male|    1|
|nm5477035|         Iola Nguyen|Female|    1|
|nm4975971|           Naj Modak|  Male|    1|
|nm1767604|         Ashok Verma|  Male|    2|
|nm2719795|              Kishan|  Male|    1|
|nm5157380|Neil Alexander Smith|  Male|    1|
|nm0576489|          Pran Mehra|  null|    1|
|nm0004434|       Shashi Kapoor|  Male|    7|
|nm5171814|            Narin Oz|Female|    1|
|nm1588131|         Shyam Arora|  Male|    2|
|nm1649656|   Chandu Allahabadi|  Male|    2|
|nm0707271|         Jagdish Raj|  Male|   11|
|nm5221129|      Jasmine Jardot|Female|    1|
|nm4874389|     Vicky K. Foster|Female|    1|
|nm7150152|        Abbie Murphy|Female|    1|
|nm3275880|            Om Sahni|  Male|    1|
|nm1542555|        Master Kelly|  Male|    1|
|nm3157251|              Naseem|Female|    1|
|nm7076286|      Katy Kartwheel|Fe

# Below lines are additional and practice 

In [41]:
val actor_pids_grp_by=dir_pid_count2.join(dir_pid_count2
                                  .toDF(dir_pid_count2.columns.map(_ + "_R"):_*),$"PID"===$"PID_R")
.dropDuplicates()

actor_pids_grp_by = [DPID: string, PID: string ... 4 more fields]


[DPID: string, PID: string ... 4 more fields]

In [42]:
actor_pids_grp_by.sort("PID").show(500)

+---------+---------+-----+---------+---------+-------+
|     DPID|      PID|count|   DPID_R|    PID_R|count_R|
+---------+---------+-----+---------+---------+-------+
|nm0496746|nm0000002|    1|nm0496746|nm0000002|      1|
|nm0000180|nm0000027|    1|nm0000180|nm0000027|      1|
|nm0896533|nm0000039|    1|nm0896533|nm0000039|      1|
|nm0896533|nm0000042|    1|nm0896533|nm0000042|      1|
|nm0004292|nm0000047|    1|nm0004292|nm0000047|      1|
|nm0485943|nm0000073|    1|nm0485943|nm0000073|      1|
|nm0000229|nm0000076|    1|nm0000229|nm0000076|      1|
|nm0178997|nm0000092|    1|nm0178997|nm0000092|      1|
|nm0000269|nm0000093|    1|nm0000269|nm0000093|      1|
|nm0149446|nm0000096|    1|nm0113819|nm0000096|      1|
|nm0113819|nm0000096|    1|nm0113819|nm0000096|      1|
|nm0149446|nm0000096|    1|nm0149446|nm0000096|      1|
|nm0113819|nm0000096|    1|nm0149446|nm0000096|      1|
|nm0001060|nm0000101|    1|nm0001060|nm0000101|      1|
|nm0583292|nm0000112|    1|nm0583292|nm0000112| 

In [43]:
val maxCount=actor_pids_grp_by.withColumn("max",max("count_R") over Window.partitionBy("PID"))

maxCount = [DPID: string, PID: string ... 5 more fields]


[DPID: string, PID: string ... 5 more fields]

In [141]:
maxCount.sort("PID").show(100)

+---------+---------+-----+---------+---------+-------+---+
|     DPID|      PID|count|   DPID_R|    PID_R|count_R|max|
+---------+---------+-----+---------+---------+-------+---+
|nm0496746|nm0000002|    1|nm0496746|nm0000002|      1|  1|
|nm0000180|nm0000027|    1|nm0000180|nm0000027|      1|  1|
|nm0896533|nm0000039|    1|nm0896533|nm0000039|      1|  1|
|nm0896533|nm0000042|    1|nm0896533|nm0000042|      1|  1|
|nm0004292|nm0000047|    1|nm0004292|nm0000047|      1|  1|
|nm0485943|nm0000073|    1|nm0485943|nm0000073|      1|  1|
|nm0000229|nm0000076|    1|nm0000229|nm0000076|      1|  1|
|nm0178997|nm0000092|    1|nm0178997|nm0000092|      1|  1|
|nm0000269|nm0000093|    1|nm0000269|nm0000093|      1|  1|
|nm0149446|nm0000096|    1|nm0149446|nm0000096|      1|  1|
|nm0113819|nm0000096|    1|nm0113819|nm0000096|      1|  1|
|nm0149446|nm0000096|    1|nm0113819|nm0000096|      1|  1|
|nm0113819|nm0000096|    1|nm0149446|nm0000096|      1|  1|
|nm0001060|nm0000101|    1|nm0001060|nm0

In [150]:
maxCount.filter($"DPID_R"==="nm0007181" && $"count_R"===$"max")
.sort("count_R")
.dropDuplicates("PID_R","max").sort(desc("count_R")).show(1000)

+---------+---------+-----+---------+---------+-------+---+
|     DPID|      PID|count|   DPID_R|    PID_R|count_R|max|
+---------+---------+-----+---------+---------+-------+---+
|nm0660967|nm0707271|    1|nm0007181|nm0707271|     11| 11|
|nm0706800|nm0471443|    1|nm0007181|nm0471443|     10| 10|
|nm0894340|nm0407002|    1|nm0007181|nm0407002|      9|  9|
|nm0412465|nm0004434|    1|nm0007181|nm0004434|      7|  7|
|nm0014180|nm0716851|    1|nm0007181|nm0716851|      5|  5|
|nm0422552|nm0347901|    1|nm0007181|nm0347901|      5|  5|
|nm0728263|nm0433945|    1|nm0007181|nm0433945|      4|  4|
|nm0814343|nm0755087|    1|nm0007181|nm0755087|      4|  4|
|nm0611582|nm0802183|    1|nm0007181|nm0802183|      4|  4|
|nm0451211|nm9036653|    1|nm0007181|nm9036653|      3|  3|
|nm0611531|nm0158332|    2|nm0007181|nm0158332|      3|  3|
|nm0007147|nm7760187|    2|nm0007181|nm7760187|      3|  3|
|nm0007181|nm0595923|    3|nm0007181|nm0595923|      3|  3|
|nm0006765|nm0756378|    1|nm0007181|nm0

In [108]:
val final_actors=persons.join(dir_pid_count,"PID")

final_actors = [PID: string, index: int ... 2 more fields]


[PID: string, index: int ... 2 more fields]

In [112]:
final_actors.printSchema
final_actors.sort("PID").show(5)

root
 |-- PID: string (nullable = true)
 |-- index: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)

+---------+-----+----------------+------+
|      PID|index|            Name|Gender|
+---------+-----+----------------+------+
|nm0000821| 1012|Amitabh Bachchan|  Male|
|nm0002043| 5417|   Madhuri Dixit|Female|
|nm0004109| 7918|  Gulshan Grover|  Male|
|nm0004334| 5375|           Rekha|Female|
|nm0004429| 6396|      Dharmendra|  Male|
+---------+-----+----------------+------+
only showing top 5 rows



In [62]:
//dir_pid_count.printSchema
//dir_pid_count.schema
//var h=dir_pid_count.schema
var modidfiedSchema=dir_pid_count.schema.map{
    case StructField(fieldName,typeName,isNullable,m) if(fieldName=="count") => StructField(fieldName,typeName,!isNullable,m)
    case x:StructField=>x
}
dir_pid_count.schema

modidfiedSchema = List(StructField(DPID,StringType,true), StructField(PID,StringType,true), StructField(count,LongType,true), StructField(yash_count,LongType,true))


StructType(StructField(DPID,StringType,true), StructField(PID,StringType,true), StructField(count,LongType,false), StructField(yash_count,LongType,true))

Now we need to compare the yash and actor count with other director with actor count (hint: self join with renamed columns)

In [85]:
//val actor_pids=dir_pid_count.join(dir_pid_count.toDF(dir_pid_count.columns.map(_ + "_R"):_*),$"PID"===$"PID_R")//.cache()
val actor_pids=dir_pid_count.join(dir_pid_count
                                  .toDF(dir_pid_count.columns.map(_ + "_R"):_*),$"PID"===$"PID_R")
.dropDuplicates()//.cache()

actor_pids = [DPID: string, PID: string ... 6 more fields]


[DPID: string, PID: string ... 6 more fields]

sample data

In [75]:
actor_pids.printSchema
actor_pids.count()

root
 |-- DPID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- count: long (nullable = true)
 |-- yash_count: long (nullable = true)
 |-- DPID_R: string (nullable = true)
 |-- PID_R: string (nullable = true)
 |-- count_R: long (nullable = true)
 |-- yash_count_R: long (nullable = true)



1151662

In [91]:
println("following  code gives error since condition1 nullable with nullable and condition2 nullable with non nullable gives error in spark sql")
actor_pids.createOrReplaceTempView("actor_pids")
spark.sql("select * from actor_pids where PID=PID_R and yash_count>=count_R and (PID is not null and PID_R is not null) order by PID").show(1000)

following  code gives error since condition1 nullable with nullable and condition2 nullable with non nullable gives error in spark sql
+---------+---------+-----+----------+---------+---------+-------+------------+
|     DPID|      PID|count|yash_count|   DPID_R|    PID_R|count_R|yash_count_R|
+---------+---------+-----+----------+---------+---------+-------+------------+
|nm0007181|nm0000821|    6|         6|nm0841918|nm0000821|      1|           0|
|nm0007181|nm0000821|    6|         6|nm1200832|nm0000821|      1|           0|
|nm0007181|nm0000821|    6|         6|nm0080180|nm0000821|      1|           0|
|nm0007181|nm0000821|    6|         6|nm0244876|nm0000821|      1|           0|
|nm0007181|nm0000821|    6|         6|nm0707494|nm0000821|      1|           0|
|nm0007181|nm0000821|    6|         6|nm0482232|nm0000821|      3|           0|
|nm0007181|nm0000821|    6|         6|nm0332950|nm0000821|      2|           0|
|nm0007181|nm0000821|    6|         6|nm0708001|nm0000821|      2

In [93]:
spark.sql("select * from actor_pids where PID='nm0000821'").sort("PID").show(100)

+---------+---------+-----+----------+---------+---------+-------+------------+
|     DPID|      PID|count|yash_count|   DPID_R|    PID_R|count_R|yash_count_R|
+---------+---------+-----+----------+---------+---------+-------+------------+
|nm1334468|nm0000821|    1|         0|nm1334468|nm0000821|      1|           0|
|nm1334468|nm0000821|    1|         0|nm0315916|nm0000821|      1|           0|
|nm1334468|nm0000821|    1|         0|nm1018493|nm0000821|      2|           0|
|nm1334468|nm0000821|    1|         0|nm0049416|nm0000821|      1|           0|
|nm1334468|nm0000821|    1|         0|nm0014180|nm0000821|      1|           0|
|nm1334468|nm0000821|    1|         0|nm3797575|nm0000821|      1|           0|
|nm1334468|nm0000821|    1|         0|nm0890060|nm0000821|      7|           0|
|nm1334468|nm0000821|    1|         0|nm0000965|nm0000821|      1|           0|
|nm1334468|nm0000821|    1|         0|nm0841918|nm0000821|      1|           0|
|nm1334468|nm0000821|    1|         0|nm

In [82]:
val yash_pids=actor_pids.filter($"PID"===$"PID_R" && $"yash_count">=$"count_R")
//.select("PID").dropDuplicates("PID")
//.sort("PID")
yash_pids
.sort("PID")
.show()
//.count()

lastException = null


Name: java.util.NoSuchElementException
Message: None.get
StackTrace:   at scala.None$.get(Option.scala:347)
  at scala.None$.get(Option.scala:345)
  at org.apache.spark.sql.execution.exchange.EnsureRequirements$$anonfun$reorder$1.apply(EnsureRequirements.scala:238)
  at org.apache.spark.sql.execution.exchange.EnsureRequirements$$anonfun$reorder$1.apply(EnsureRequirements.scala:233)
  at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
  at org.apache.spark.sql.execution.exchange.EnsureRequirements.reorder(EnsureRequirements.scala:233)
  at org.apache.spark.sql.execution.exchange.EnsureRequirements.reorderJoinKeys(EnsureRequirements.scala:256)
  at org.apache.spark.sql.execution.exchange.EnsureRequirements.org$apache$spark$sql$execution$exchange$EnsureRequirements$$reorderJoinPredicates(EnsureRequirements.scala:289)
  at org.apache.spark.sql.execution.exchange.EnsureRequirements$$anonfu

In [135]:
val yash_pids2=yash_pids.select("PID","count").dropDuplicates("PID").join(persons,"PID").dropDuplicates()
yash_pids2.sort("PID").show(10)

+---------+-----+-----+----------------+------+
|      PID|count|index|            Name|Gender|
+---------+-----+-----+----------------+------+
|nm0000821|    6| 1012|Amitabh Bachchan|  Male|
|nm0004109|    2| 7918|  Gulshan Grover|  Male|
|nm0004434|    7|15980|   Shashi Kapoor|  Male|
|nm0004435|    3| 3456|   Rajesh Khanna|  Male|
|nm0004437|    2| 4377|         Sridevi|Female|
|nm0004487|    2| 2274|     Juhi Chawla|Female|
|nm0004564|    4| 6628|     Hema Malini|Female|
|nm0004570|    2| 6400|      Sunil Dutt|  Male|
|nm0006348|    2|11760|  Rajendra Kumar|  Male|
|nm0006369|    2|14541|     Ashok Kumar|  Male|
+---------+-----+-----+----------------+------+
only showing top 10 rows



yash_pids2 = [PID: string, count: bigint ... 3 more fields]


[PID: string, count: bigint ... 3 more fields]

In [136]:
dir_pid_count.where($"PID"==="nm0000821").sort("PID").show()

+---------+---------+-----+----------+
|     DPID|      PID|count|yash_count|
+---------+---------+-----+----------+
|nm1334468|nm0000821|    1|        -1|
|nm0315916|nm0000821|    1|        -1|
|nm1018493|nm0000821|    2|        -1|
|nm0049416|nm0000821|    1|        -1|
|nm0014180|nm0000821|    1|        -1|
|nm3797575|nm0000821|    1|        -1|
|nm0890060|nm0000821|    7|        -1|
|nm0841918|nm0000821|    1|        -1|
|nm0000965|nm0000821|    1|        -1|
|nm0474859|nm0000821|    4|        -1|
|nm0348495|nm0000821|    1|        -1|
|nm0759662|nm0000821|    2|        -1|
|nm1084488|nm0000821|    1|        -1|
|nm0220828|nm0000821|    8|        -1|
|nm0025629|nm0000821|    1|        -1|
|nm1224879|nm0000821|    2|        -1|
|nm0004466|nm0000821|    1|        -1|
|nm0660967|nm0000821|    1|        -1|
|nm1999473|nm0000821|    1|        -1|
|nm0422689|nm0000821|    1|        -1|
+---------+---------+-----+----------+
only showing top 20 rows



In [44]:
val yash_pids_less=actor_pids.filter($"PID"===$"PID_R").filter($"yash_count"<$"count_R")
//.select("PID").dropDuplicates("PID")
//.sort("PID")
yash_pids_less
.sort("PID")
//.show()
.count()
//total=1151662=(1145024+2408)

yash_pids_less = [DPID: string, PID: string ... 6 more fields]


1145024

In [45]:
yash_pids.join(yash_pids_less,"PID").count()

12720175

In [128]:
println("below code giving error, if apply filter one by one working, together not working")
//below code giving error, if apply filter one by one working, together not working
//actor_pids.filter($"PID"===$"PID_R" && $"yash_count">=$"count_R")
//.select("PID").dropDuplicates("PID")
//.sort("PID")
//.show()
//.count()

below code giving error, if apply filter one by one working, together not working


In [49]:
val dir_pid_count_yash=dir_pid_count
.filter("yash_count>=count and yash_count!=0")
.sort(desc("yash_count"))

dir_pid_count_yash = [DPID: string, PID: string ... 2 more fields]


[DPID: string, PID: string ... 2 more fields]

In [50]:
dir_pid_count_yash.count()

430

In [51]:
dir_pid_count_yash.show(40)

+---------+---------+-----+----------+
|     DPID|      PID|count|yash_count|
+---------+---------+-----+----------+
|nm0007181|nm0707271|   11|        11|
|nm0007181|nm0471443|   10|        10|
|nm0007181|nm0407002|    9|         9|
|nm0007181|nm0025630|    8|         8|
|nm0007181|nm0534501|    8|         8|
|nm0007181|nm0451600|    7|         7|
|nm0007181|nm0004434|    7|         7|
|nm0007181|nm0000821|    6|         6|
|nm0007181|nm0716851|    5|         5|
|nm0007181|nm0347901|    5|         5|
|nm0007181|nm0802183|    4|         4|
|nm0007181|nm0894340|    4|         4|
|nm0007181|nm0433945|    4|         4|
|nm0007181|nm0451321|    4|         4|
|nm0007181|nm0755087|    4|         4|
|nm0007181|nm0004564|    4|         4|
|nm0007181|nm0438501|    4|         4|
|nm0007181|nm0438463|    3|         3|
|nm0007181|nm9036653|    3|         3|
|nm0007181|nm0159165|    3|         3|
|nm0007181|nm7760187|    3|         3|
|nm0007181|nm0438465|    3|         3|
|nm0007181|nm0595923|    

In [52]:
dir_pid_count.filter(!col("PID").isin(dir_pid_count.filter("count>yash_count").select("PID").take(10)))
.sort(desc("count")).show()

Name: org.apache.spark.sql.AnalysisException
Message: Unsupported component type interface org.apache.spark.sql.Row in arrays;
StackTrace:   at org.apache.spark.sql.catalyst.expressions.Literal$.componentTypeToDataType(literals.scala:117)
  at org.apache.spark.sql.catalyst.expressions.Literal$.apply(literals.scala:70)
  at org.apache.spark.sql.catalyst.expressions.Literal$$anonfun$create$2.apply(literals.scala:164)
  at org.apache.spark.sql.catalyst.expressions.Literal$$anonfun$create$2.apply(literals.scala:164)
  at scala.util.Try.getOrElse(Try.scala:79)
  at org.apache.spark.sql.catalyst.expressions.Literal$.create(literals.scala:163)
  at org.apache.spark.sql.functions$.typedLit(functions.scala:127)
  at org.apache.spark.sql.functions$.lit(functions.scala:110)
  at org.apache.spark.sql.Column$$anonfun$isin$1.apply(Column.scala:796)
  at org.apache.spark.sql.Column$$anonfun$isin$1.apply(Column.scala:796)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala

In [53]:
dir_pid_count.filter("PID in ('nm0707271','nm0471443') and count>yash_count").sort(desc("count")).show()

+---------+---------+-----+----------+
|     DPID|      PID|count|yash_count|
+---------+---------+-----+----------+
|nm0220828|nm0707271|    6|         0|
|nm0159148|nm0707271|    5|         0|
|nm0149850|nm0707271|    5|         0|
|nm0159148|nm0471443|    4|         0|
|nm0025629|nm0707271|    4|         0|
|nm0451846|nm0707271|    4|         0|
|nm0159162|nm0707271|    3|         0|
|nm0109279|nm0707271|    3|         0|
|nm0712541|nm0707271|    3|         0|
|nm0080332|nm0471443|    2|         0|
|nm0759662|nm0471443|    2|         0|
|nm0007131|nm0707271|    2|         0|
|nm0346618|nm0707271|    2|         0|
|nm0024039|nm0707271|    2|         0|
|nm0576454|nm0707271|    2|         0|
|nm0849200|nm0707271|    2|         0|
|nm0025608|nm0707271|    2|         0|
|nm0755997|nm0471443|    2|         0|
|nm0066066|nm0707271|    2|         0|
|nm0463319|nm0707271|    2|         0|
+---------+---------+-----+----------+
only showing top 20 rows



lastException: Throwable = null


In [54]:
dir_pid_count.filter("PID!='nm0007181' and count>yash_count").sort(desc("count")).show()

+---------+---------+-----+----------+
|     DPID|      PID|count|yash_count|
+---------+---------+-----+----------+
|nm0223522|nm0456094|   23|         0|
|nm0223522|nm0007106|   20|         0|
|nm0223522|nm0434318|   20|         0|
|nm0080315|nm0318622|   19|         0|
|nm0223522|nm0332871|   17|         0|
|nm0698184|nm0712546|   16|         0|
|nm0698184|nm2147526|   15|         0|
|nm0223522|nm0442479|   14|         0|
|nm0223522|nm0451600|   14|         0|
|nm0080315|nm0451272|   14|         0|
|nm0007147|nm0007147|   13|         0|
|nm0764316|nm0451864|   13|         0|
|nm0223522|nm0025630|   13|         0|
|nm0223522|nm0794510|   12|         0|
|nm0759662|nm0534501|   12|         0|
|nm0698184|nm1056425|   12|         0|
|nm0122216|nm0505323|   12|         0|
|nm0080315|nm0080324|   12|         0|
|nm0223522|nm2136994|   11|         0|
|nm0223522|nm0848308|   11|         0|
+---------+---------+-----+----------+
only showing top 20 rows



In [55]:
val actor_with_director=

Name: Syntax Error.
Message: 
StackTrace: 

In [56]:
movies.show(1)

+-----+---------+------+----+------+---------+
|index|      MID| title|year|rating|num_votes|
+-----+---------+------+----+------+---------+
| 3008|tt0036077|Kismet|1943|   7.5|       71|
+-----+---------+------+----+------+---------+
only showing top 1 row



In [57]:
val names=m_cast.filter("MID='tt0036077'").select("PID").show()//.join(persons,"")

+---------+
|      PID|
+---------+
|nm0695129|
|nm0576454|
|nm0747083|
|nm0220847|
|nm0685693|
|nm1593292|
|nm0151537|
|nm0623180|
|nm1117384|
|nm0610799|
|nm0006369|
|nm0788396|
|nm0008932|
|nm0474926|
+---------+



names: Unit = ()


In [58]:
val yc_pid=persons.filter("Name is not null")
.filter(p=>p(2).toString.toLowerCase.equals("yash chopra")).select("PID").show()
println(yc_pid)

+---------+
|      PID|
+---------+
|nm0007181|
+---------+

()


yc_pid: Unit = ()


In [59]:
val yc_pid_mids=m_directors.filter("PID='nm0007181'").select("MID").dropDuplicates("MID").show(10000000)

+---------+
|      MID|
+---------+
|tt0107777|
|tt0083081|
|tt0096390|
|tt0085912|
|tt0064506|
|tt0109555|
|tt0059893|
|tt0097045|
|tt0158585|
|tt0102258|
|tt2176013|
|tt0052736|
|tt0118983|
|tt0158690|
|tt0158449|
|tt0072860|
|tt0074730|
|tt0150251|
|tt0420332|
|tt0078418|
|tt0079386|
+---------+



yc_pid_mids: Unit = ()


In [60]:
persons.show(1)

+-----+---------+------+------+
|index|      PID|  Name|Gender|
+-----+---------+------+------+
|15793|nm0022343|Alpana|Female|
+-----+---------+------+------+
only showing top 1 row



In [61]:
val yc=persons.filter("Name is not null").show()

+-----+---------+------------------+------+
|index|      PID|              Name|Gender|
+-----+---------+------------------+------+
|15793|nm0022343|            Alpana|Female|
| 3121|nm0028416|     Martin Andris|  Male|
|25039|nm0045140|     Sikharani Bag|Female|
|26817|nm0066065|     Mohinder Bedi|  Male|
|37686|nm0080157|     Devika Bhagat|  null|
| 2729|nm0080335|Paul Bhattacharjee|  Male|
| 6649|nm0091035|        Kim Bodnia|  Male|
|14277|nm0149234|           Chandni|Female|
|15514|nm0151539|    Chandrashekhar|  Male|
|27622|nm0157262| Kallu Chidambaram|  Male|
| 1080|nm0161419|  Candus Churchill|Female|
|30400|nm0201911|        Saurav Das|  Male|
|22843|nm0432340|            Julián|  Male|
|23816|nm0438497|      Rajiv Kapoor|  Male|
| 4983|nm0443365|       Nathan Kaye|  Male|
| 7983|nm0525472|       Man-Wai Luk|  Male|
|22865|nm0538303|   Sreela Majumdar|Female|
|30195|nm0540027|           Mallika|Female|
|21092|nm0576492|      Rakesh Mehra|  null|
|35083|nm0631539|   Govind Nihal

yc: Unit = ()
